In [1]:
from lxml import html
import requests
import re
import tqdm

In [2]:
#zaken = ['ECLI:NL:GHDHA:2017:2572', 'ECLI:NL:GHAMS:2017:3031', 'ECLI:NL:PHR:2014:1901', 'ECLI:NL:GHSHE:2017:1760']
with open('../data/eclis.csv') as f:
    zaken = f.read().splitlines()

In [3]:
def scrape_zaak(eclinummer):
    results=[]
    page = None
    tries = 0
    for i in range(10):
        try:
            page = requests.get('https://uitspraken.rechtspraak.nl/inziendocument?id={ecli}'.format(ecli=eclinummer))
            break
        except:
            pass
        
    if page is None:
        print("FAILED AFTER 10 TRIES", eclinummer)
        return []

    htmltree = html.fromstring(page.content)
    tree = htmltree.xpath('//dt[text()="Formele relaties"]/following-sibling::dd[1]//text()')
    tree = [' '.join(i.split()) for i in tree]
    tree = [i for i in tree if i]
    is_ecli = [s.startswith("ECLI") for s in tree]
    for i in range(0, len(is_ecli)):
        if is_ecli[i] is True:
            c_ecli = tree[i]
            b_ecli = ''
            n_ecli = ''
            if i>0:
                if is_ecli[i-1] is False:
                    b_ecli = tree[i-1]
            if i == len(is_ecli)-2:
                if is_ecli[i+1] is False:
                    n_ecli = tree[i+1]
            if i < len(is_ecli)-2:
                if is_ecli[i+2] is False and is_ecli[i+1] is False:
                    n_ecli = tree[i+1]
            result = (eclinummer, c_ecli, b_ecli, n_ecli)
            results.append(result)
    return results

In [4]:
#from multiprocessing.pool import ThreadPool as Pool
from multiprocessing import Pool

def scrape_zaken(zaken):
    results = []
    for zaak in tqdm.tqdm(zaken):
        results.extend(scrape_zaak(zaak))
    return results

In [ ]:
from multiprocessing.pool import ThreadPool
p = Pool(32)
result = list(tqdm.tqdm(p.imap_unordered(scrape_zaak, zaken), total=len(zaken)))

import pickle
with open('../data/bas_output.pkl', 'w') as f:
    pickle.dump(result, f)

 57%|█████▋    | 51470/89999 [47:49<31:24, 20.44it/s]  

In [ ]:
with open('../data/bas_output.pkl', 'r') as f:
    print(pickle.load(f))